In [1]:
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import os

import time
import torch.nn as nn
import matplotlib.pyplot as plt
import cv2 
import itertools

import csv
import torch
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data.sampler import SubsetRandomSampler


from vgg16_2 import VGG_16_2


import argparse
from data_loader_half_1 import get_loader
from data_loader_half_1  import get_dataset

import math

import numpy as np
import torch
import torch.nn as nn
import torch.utils.data
import torch.utils.data.distributed
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torchvision.models as models
import time
import shutil
import os
import torch.nn.functional as F

# Loading dataset

In [5]:
os.environ['CUDA_VISIBLE_DEVICES']='0'
celeba_image_dir = '/home/name/celeba/stargan/data/celeba/images'
attr_path = '/home/name/celeba/stargan/data/celeba/list_attr_celeba.txt'
id_path = '/home/name/celeba/stargan/data/celeba/identity_CelebA.txt'
selected_attrs = ['Black_Hair']
celeba_crop_size = 224
image_size = 224 
batch_size = 30
num_workers = 0
mode1 = 'train' #test or train
mode2 = 'test'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") #"cuda:0" if torch.cuda.is_available() else 
torch.set_default_tensor_type('torch.cuda.FloatTensor')


 



In [6]:
celeba_loader_train, data_train = get_loader(celeba_image_dir, attr_path, id_path, selected_attrs,celeba_crop_size, image_size, batch_size,'CelebA', mode1, num_workers)
celeba_loader_test, data_test = get_loader(celeba_image_dir, attr_path, id_path, selected_attrs,celeba_crop_size, image_size, batch_size,'CelebA', mode2, num_workers)

Finished preprocessing the CelebA dataset...
Finished preprocessing the CelebA dataset...


In [7]:
celeba_loader_test2, data_test2 = get_loader(celeba_image_dir, attr_path, id_path, selected_attrs,celeba_crop_size, image_size, 1,'CelebA', mode2, num_workers)

Finished preprocessing the CelebA dataset...


# Validate

In [8]:
def validate(model):
    model.to(device)
    model.eval()
    correct = 0
    size =0
    for im, labels in celeba_loader_test:
        im, labels = im.to(device), labels.to(device)
        labels = labels.long().squeeze(1)
        preds, _ = model(im)
        values, indices = preds.max(-1)
        correct += indices.eq(labels).sum()
        
        del im
        del preds
        
        del values
        
        print(correct)
        size += len(labels)
        del labels
        print(float(correct)/float(size))
        
    print(size)
    print(correct)


In [9]:
def validate_per_example(model, mask):
    model.to(device)
    model.eval()
    correct = 0
    size =0
    for im, labels in celeba_loader_test2:
        im, labels = im.to(device), labels[:,0].to(device)
        labels = labels.long()
        preds, _ = model(im, mask)
        values, indices = preds.max(-1)
        correct += indices.eq(labels).sum()

        
        del im
        del preds
        
        del values
        
        size += len(labels)
        del labels
        print(float(correct)/float(size))
        
    print(size)
    print(correct)
    return size,correct



In [10]:
def validate_per_example_mask(model, mask):
    model.to(device)
    model.eval()
    correct = 0
    size =0
    for im, labels in celeba_loader_test2:
        im, labels = im.to(device), labels.to(device)
        labels = labels.long().squeeze(1)
        im = im
        preds, _ = model(im,mask)
        values, indices = preds.max(-1)
        correct += indices.eq(labels).sum()

        
        del im
        del preds
        
        del values
        
        size += len(labels)
        del labels
        print(float(correct)/float(size))
        
    print(size)
    print(correct)
    return size,correct



# Train

In [25]:
def train(model, criterion, optimizer, mask, increase= False, coef=1):
    model.to(device)
    model.train()
    size = 0
    correcta = 0
    for i, (im, labels) in enumerate(celeba_loader_train):
        #if (i ==1):
         #   print (labels)
        print("iteration no", i)
        im, labels = im.to(device), labels[:,0].to(device)
        labels = labels.long()
        
        
        
        optimizer.zero_grad()
        output,noisy = model(im,mask)
        
        
        print(torch.mean(model.intermed.scales()))
        print(torch.max(model.intermed.scales()))
        print(torch.min(model.intermed.scales()))
        
        if (increase):
            #loss = criterion(output, labels) + coef* 1/(torch.mean(model.intermed.scales()) )

            loss = criterion(output, labels) - coef* torch.log(torch.mean(model.intermed.scales()) ) + 10
            
        else:
            loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        print (loss, "loss")
        #preds = model(im)
        values, indices = output.max(-1)
        size += len(labels)
        correcta += indices.eq(labels).sum()
        correct = indices.eq(labels).sum()
        del im
        del labels
        del output
        del values
        del indices
        
        
        print(float(correcta)/float(size))
            
            
    print(correcta, "correct out of all")
    print(size)
    return correct, size
     
        





In [26]:
def train_mask(model, criterion, optimizer, mask, increase= False, coef=1):
    model.to(device)
    model.train()
    size = 0
    correcta = 0
    for i, (im, labels) in enumerate(celeba_loader_train):
        #if (i ==1):
         #   print (labels)
        print("iteration no", i)
        im, labels = im.to(device), labels[:,0].to(device)
        labels = labels.long()
        
        
        im = im 
        optimizer.zero_grad()
        output,noisy = model(im,mask)
        
        
        print(torch.mean(model.intermed.scales()))
        print(torch.max(model.intermed.scales()))
        print(torch.min(model.intermed.scales()))
        
        if (increase):

            #loss = criterion(output, labels) + coef* 1/(torch.mean(model.intermed.scales()) )
            loss = criterion(output, labels) - coef* (torch.mean(model.intermed.scales()) ) + 10
        else:
            loss = criterion(output, labels)
        loss.backward()
        optimizer.step()
        print (loss, "loss")
        #preds = model(im)
        values, indices = output.max(-1)
        size += len(labels)
        correcta += indices.eq(labels).sum()
        correct = indices.eq(labels).sum()
        del im
        del labels
        del output
        del values
        del indices
        
        
        print(float(correcta)/float(size))
            
            
    print(correcta, "correct out of all")
    print(size)
    return correct, size
     
        





# Noisy Network

In [27]:

class NoisyActivation(nn.Module):
    def __init__(self,  given_locs, given_scales, min_scale, max_scale):
        super(NoisyActivation, self).__init__()
        size = given_scales.shape
        self.min_scale = min_scale
        self.max_scale = max_scale
        self.given_locs = given_locs 
        self.given_scales = given_scales
        self.locs = nn.Parameter(torch.Tensor(size).copy_(self.given_locs))         
        self.rhos = nn.Parameter(torch.ones(size)-5) #-inf


        self.normal = torch.distributions.normal.Normal(0,1)
        self.rhos.requires_grad = False
        self.locs.requires_grad = False
        
    def scales(self):
        return (1.0 +torch.tanh(self.rhos))/2*(self.max_scale-self.min_scale) +self.min_scale             
    
    def sample_noise(self, mask):
        epsilon = self.normal.sample(self.rhos.shape)
        return self.locs + self.scales() * epsilon*mask
                                 
                               
                            
    def forward(self, input, mask):
        noise = self.sample_noise(mask)
        return (input)*mask + noise



class vgg_syn(nn.Module):

    def __init__(self, model, min_scale,max_scale, given_locs, given_scale):
        super(vgg_syn, self).__init__()
        

        self.intermed = NoisyActivation( given_locs, given_scale, min_scale, max_scale)
        self.model_pt3 = model

        
        for param in self.model_pt3.parameters():
            param.requires_grad = False

                
        self.intermed.rhos.requires_grad = True
        self.intermed.locs.requires_grad = True
                                 
    def forward(self, img, mask):
                                 
        
        x = self.intermed(img, mask)
        noisy = x.detach()
         

        x = self.model_pt3(x)                                 

        return x, noisy
    
                                 
 






In [28]:

model = models.resnet18(num_classes=2)

model.load_state_dict(torch.load("resnet-hair-90l0",  map_location=device))


criterion = nn.NLLLoss()
mus = torch.zeros((3,224,224))
scale = torch.ones((3,224,224))*0.001
model_syn = vgg_syn(model ,0.0,5 ,mus, scale )
model_syn.load_state_dict(torch.load("resnet-hair-avg-cloak", map_location=device))


<All keys matched successfully>

In [31]:
#mult_mask =  torch.ones(model_syn.intermed.rhos.shape)
mult_mask = torch.where(model_syn.intermed.scales()>4.5, torch.zeros(model_syn.intermed.scales().shape), torch.ones(model_syn.intermed.scales().shape))

In [32]:
criterion = nn.CrossEntropyLoss().to(device)

In [33]:
#model_syn.intermed.rhos.requires_grad = False

In [34]:
d = list(filter(lambda p: p.requires_grad, model_syn.parameters()))
print(len(d))

2


In [36]:
lr =  0.001 
for epoch in range (1): # 2 epoch coef 1 lr 0.01real
   
    optimizer = optim.Adam(filter(lambda p: p.requires_grad, model_syn.parameters()), lr=lr, weight_decay=0)
    cor, size = train(model_syn, criterion, optimizer, mult_mask,True, 2)
    print (cor,size)
 
    validate_single_label(model_syn, mask)
    



iteration no 0
tensor(4.2589, grad_fn=<MeanBackward0>)
tensor(5.0000, grad_fn=<MaxBackward1>)
tensor(3.6955e-05, grad_fn=<MinBackward1>)
tensor(7.4449, grad_fn=<AddBackward0>) loss
0.8666666666666667
iteration no 1
tensor(4.2589, grad_fn=<MeanBackward0>)
tensor(5.0000, grad_fn=<MaxBackward1>)
tensor(3.7104e-05, grad_fn=<MinBackward1>)
tensor(7.2842, grad_fn=<AddBackward0>) loss
0.9166666666666666
iteration no 2
tensor(4.2589, grad_fn=<MeanBackward0>)
tensor(5.0000, grad_fn=<MaxBackward1>)
tensor(3.7104e-05, grad_fn=<MinBackward1>)
tensor(7.4385, grad_fn=<AddBackward0>) loss
0.8666666666666667
iteration no 3
tensor(4.2589, grad_fn=<MeanBackward0>)
tensor(5.0000, grad_fn=<MaxBackward1>)
tensor(3.7104e-05, grad_fn=<MinBackward1>)
tensor(7.4670, grad_fn=<AddBackward0>) loss
0.8583333333333333
iteration no 4
tensor(4.2590, grad_fn=<MeanBackward0>)
tensor(5.0000, grad_fn=<MaxBackward1>)
tensor(3.7253e-05, grad_fn=<MinBackward1>)
tensor(7.4308, grad_fn=<AddBackward0>) loss
0.8533333333333334


tensor(4.2591, grad_fn=<MeanBackward0>)
tensor(5.0000, grad_fn=<MaxBackward1>)
tensor(3.7402e-05, grad_fn=<MinBackward1>)
tensor(7.4135, grad_fn=<AddBackward0>) loss
0.834108527131783
iteration no 43
tensor(4.2591, grad_fn=<MeanBackward0>)
tensor(5.0000, grad_fn=<MaxBackward1>)
tensor(3.7402e-05, grad_fn=<MinBackward1>)
tensor(7.5447, grad_fn=<AddBackward0>) loss
0.8333333333333334
iteration no 44
tensor(4.2591, grad_fn=<MeanBackward0>)
tensor(5.0000, grad_fn=<MaxBackward1>)
tensor(3.7402e-05, grad_fn=<MinBackward1>)
tensor(7.3135, grad_fn=<AddBackward0>) loss
0.8348148148148148
iteration no 45
tensor(4.2591, grad_fn=<MeanBackward0>)
tensor(5.0000, grad_fn=<MaxBackward1>)
tensor(3.7402e-05, grad_fn=<MinBackward1>)
tensor(7.4678, grad_fn=<AddBackward0>) loss
0.8333333333333334
iteration no 46
tensor(4.2592, grad_fn=<MeanBackward0>)
tensor(5.0000, grad_fn=<MaxBackward1>)
tensor(3.7402e-05, grad_fn=<MinBackward1>)
tensor(7.7299, grad_fn=<AddBackward0>) loss
0.8312056737588652
iteration no

iteration no 83
tensor(4.2593, grad_fn=<MeanBackward0>)
tensor(5.0000, grad_fn=<MaxBackward1>)
tensor(3.7700e-05, grad_fn=<MinBackward1>)
tensor(7.4432, grad_fn=<AddBackward0>) loss
0.8277777777777777
iteration no 84
tensor(4.2593, grad_fn=<MeanBackward0>)
tensor(5.0000, grad_fn=<MaxBackward1>)
tensor(3.7700e-05, grad_fn=<MinBackward1>)
tensor(7.4781, grad_fn=<AddBackward0>) loss
0.827843137254902
iteration no 85
tensor(4.2593, grad_fn=<MeanBackward0>)
tensor(5.0000, grad_fn=<MaxBackward1>)
tensor(3.7700e-05, grad_fn=<MinBackward1>)
tensor(7.3625, grad_fn=<AddBackward0>) loss
0.8282945736434109
iteration no 86
tensor(4.2593, grad_fn=<MeanBackward0>)
tensor(5.0000, grad_fn=<MaxBackward1>)
tensor(3.7700e-05, grad_fn=<MinBackward1>)
tensor(7.5428, grad_fn=<AddBackward0>) loss
0.828352490421456
iteration no 87
tensor(4.2593, grad_fn=<MeanBackward0>)
tensor(5.0000, grad_fn=<MaxBackward1>)
tensor(3.7849e-05, grad_fn=<MinBackward1>)
tensor(7.6534, grad_fn=<AddBackward0>) loss
0.82727272727272

iteration no 125
tensor(4.2595, grad_fn=<MeanBackward0>)
tensor(5.0000, grad_fn=<MaxBackward1>)
tensor(3.7551e-05, grad_fn=<MinBackward1>)
tensor(7.4507, grad_fn=<AddBackward0>) loss
0.8285714285714286
iteration no 126
tensor(4.2595, grad_fn=<MeanBackward0>)
tensor(5.0000, grad_fn=<MaxBackward1>)
tensor(3.7551e-05, grad_fn=<MinBackward1>)
tensor(7.5041, grad_fn=<AddBackward0>) loss
0.8278215223097113
iteration no 127
tensor(4.2595, grad_fn=<MeanBackward0>)
tensor(5.0000, grad_fn=<MaxBackward1>)
tensor(3.7551e-05, grad_fn=<MinBackward1>)
tensor(7.3607, grad_fn=<AddBackward0>) loss
0.828125
iteration no 128
tensor(4.2595, grad_fn=<MeanBackward0>)
tensor(5.0000, grad_fn=<MaxBackward1>)
tensor(3.7402e-05, grad_fn=<MinBackward1>)
tensor(7.5306, grad_fn=<AddBackward0>) loss
0.8276485788113696
iteration no 129
tensor(4.2595, grad_fn=<MeanBackward0>)
tensor(5.0000, grad_fn=<MaxBackward1>)
tensor(3.7402e-05, grad_fn=<MinBackward1>)
tensor(7.2958, grad_fn=<AddBackward0>) loss
0.8284615384615385


iteration no 166
tensor(4.2597, grad_fn=<MeanBackward0>)
tensor(5.0000, grad_fn=<MaxBackward1>)
tensor(3.7551e-05, grad_fn=<MinBackward1>)
tensor(7.3610, grad_fn=<AddBackward0>) loss
0.8323353293413174
iteration no 167
tensor(4.2597, grad_fn=<MeanBackward0>)
tensor(5.0000, grad_fn=<MaxBackward1>)
tensor(3.7551e-05, grad_fn=<MinBackward1>)
tensor(7.3624, grad_fn=<AddBackward0>) loss
0.8325396825396826
iteration no 168
tensor(4.2597, grad_fn=<MeanBackward0>)
tensor(5.0000, grad_fn=<MaxBackward1>)
tensor(3.7551e-05, grad_fn=<MinBackward1>)
tensor(7.3106, grad_fn=<AddBackward0>) loss
0.8331360946745562
iteration no 169
tensor(4.2597, grad_fn=<MeanBackward0>)
tensor(5.0000, grad_fn=<MaxBackward1>)
tensor(3.7551e-05, grad_fn=<MinBackward1>)
tensor(7.5703, grad_fn=<AddBackward0>) loss
0.8331372549019608


KeyboardInterrupt: 

In [20]:
validate_per_example(model_syn, mult_mask)

1.0
1.0
1.0
1.0
0.8
0.8333333333333334
0.8571428571428571
0.75
0.7777777777777778
0.8
0.8181818181818182
0.8333333333333334
0.8461538461538461
0.8571428571428571
0.8666666666666667
0.875
0.8823529411764706
0.8333333333333334
0.8421052631578947
0.85
0.8095238095238095
0.8181818181818182
0.8260869565217391
0.8333333333333334
0.84
0.8461538461538461
0.8518518518518519
0.8571428571428571
0.8620689655172413
0.8666666666666667
0.8709677419354839
0.875
0.8787878787878788
0.8529411764705882
0.8571428571428571
0.8611111111111112
0.8378378378378378
0.8421052631578947
0.8461538461538461
0.825
0.8048780487804879
0.7857142857142857
0.7906976744186046
0.7954545454545454
0.8
0.8043478260869565
0.8085106382978723
0.8125
0.8163265306122449
0.82
0.8235294117647058
0.8269230769230769
0.8301886792452831
0.8333333333333334
0.8363636363636363
0.8392857142857143
0.8421052631578947
0.8448275862068966
0.847457627118644
0.85
0.8524590163934426
0.8548387096774194
0.8571428571428571
0.859375
0.8615384615384616
0.

0.7852760736196319
0.7857142857142857
0.7861507128309573
0.7865853658536586
0.7849898580121704
0.7854251012145749
0.7858585858585858
0.7862903225806451
0.7867203219315896
0.7871485943775101
0.7875751503006012
0.788
0.7884231536926147
0.7888446215139442
0.7872763419483101
0.7876984126984127
0.7881188118811882
0.7885375494071146
0.7889546351084813
0.7874015748031497
0.787819253438114
0.788235294117647
0.786692759295499
0.78515625
0.783625730994152
0.7840466926070039
0.7844660194174757
0.7829457364341085
0.781431334622824
0.7818532818532818
0.7822736030828517
0.7826923076923077
0.781190019193858
0.7816091954022989
0.7820267686424475
0.7824427480916031
0.7828571428571428
0.7832699619771863
0.7836812144212524
0.7840909090909091
0.782608695652174
0.7830188679245284
0.783427495291902
0.7838345864661654
0.7842401500938087
0.7846441947565543
0.7831775700934579
0.7835820895522388
0.7839851024208566
0.7843866171003717
0.7847866419294991
0.7851851851851852
0.7855822550831792
0.7841328413284133
0.7

0.7797872340425532
0.7800212539851222
0.7802547770700637
0.7794273594909862
0.7796610169491526
0.7798941798941799
0.7801268498942917
0.7803590285110876
0.7805907172995781
0.7808219178082192
0.7810526315789473
0.7812828601472135
0.7815126050420168
0.7806925498426023
0.7809224318658281
0.7801047120418848
0.7803347280334728
0.780564263322884
0.7807933194154488
0.781021897810219
0.78125
0.7814776274713839
0.7817047817047817
0.7819314641744548
0.7821576763485477
0.7823834196891192
0.7815734989648033
0.7807652533609101
0.78099173553719
0.7812177502579979
0.7804123711340206
0.780638516992791
0.7808641975308642
0.7810894141829393
0.7813141683778234
0.7815384615384615
0.7807377049180327
0.7809621289662231
0.7811860940695297
0.7814096016343207
0.7816326530612245
0.780835881753313
0.780040733197556
0.7792472024415056
0.7794715447154471
0.7796954314720812
0.7799188640973631
0.7801418439716312
0.7793522267206477
0.7795753286147624
0.7797979797979798
0.7800201816347124
0.780241935483871
0.7804632426

0.7819225251076041
0.782078853046595
0.7815186246418339
0.7816750178954903
0.7818311874105865
0.781987133666905
0.7814285714285715
0.7815845824411135
0.7817403708987162
0.781895937277263
0.782051282051282
0.7822064056939502
0.7823613086770982
0.7818052594171997
0.7819602272727273
0.7814052519517388
0.7815602836879433
0.781715095676825
0.7818696883852692
0.7820240622788394
0.7821782178217822
0.7823321554770318
0.7824858757062146
0.7826393789696542
0.7820874471086037
0.7822410147991543
0.7816901408450704
0.7811400422237861
0.7812939521800282
0.7814476458186929
0.7816011235955056
0.7810526315789473
0.7805049088359046
0.7806587245970568
0.780812324929972
0.780965710286914
0.7804195804195804
0.779874213836478
0.7793296089385475
0.7794836008374041
0.7796373779637378
0.7797909407665505
0.7799442896935933
0.7800974251913709
0.780250347705146
0.7804030576789437
0.7798611111111111
0.7800138792505205
0.7801664355062413
0.7803187803187803
0.7804709141274239
0.7806228373702422
0.780774550484094
0.7

0.775609756097561
0.7757313109425785
0.7758527341635084
0.775974025974026
0.7760951865873446
0.7762162162162162
0.7763371150729336
0.7764578833693304
0.7765785213167836
0.7766990291262136
0.7768194070080863
0.7769396551724138
0.7765212708669897
0.7766415500538213
0.7767616998386229
0.7763440860215054
0.7764642665233745
0.7765843179377014
0.7761674718196457
0.7762875536480687
0.7758713136729223
0.7759914255091104
0.776111408677022
0.7762312633832976
0.7758159443552702
0.7759358288770053
0.7760555852485302
0.7761752136752137
0.7762947143619862
0.7764140875133404
0.7765333333333333
0.7766524520255863
0.7767714437932871
0.7768903088391906
0.77700904736562
0.776595744680851
0.7767145135566188
0.7768331562167906
0.776420605416888
0.7765392781316348
0.7761273209549071
0.7762460233297985
0.7763645998940116
0.7764830508474576
0.7766013763896241
0.7761904761904762
0.7763088313061872
0.7764270613107822
0.7765451664025357
0.776663146779303
0.7762532981530343
0.7763713080168776
0.7764891934633632
0

...

0.7824888888888889
0.7824193405030664
0.7824386775684323
0.7824580111970141
0.7824773413897281
0.7824078187472234
0.7824271499644634
0.7823576441325397
0.7823769763723575
0.7823074873434586
0.7822380106571936
0.7822573483704822
0.7822766826496181
0.7822960134955163
0.7823153409090909
0.7823346648912561
0.7823539854429257
0.7823733025650129
0.7823926162584309
0.7824119265240926
0.7824312333629104
0.7824505367757963
0.7824698367636622
0.7824891333274195
0.7825084264679795
0.7824390243902439
0.7824583185526782
0.782388933226922
0.7824082284092925
0.7823388598279989
0.7822695035460993
0.7822888041840262
0.7822194646339301
0.7822387662855623
0.782258064516129
0.7822773593265396
0.7822080453659401
0.7822273411889785
0.7822466335931963
0.782177340774205
0.7821966341895483
0.7822159241874059
0.7822352107686857
0.7822544939342956
0.7822737736851425
0.7822930500221337
0.7822237960339944
0.7822430733823139
0.7822623473181094
0.7821931144349057
0.782212389380531
0.7821431731705158
0.78216244912404

0.7820249700701214
0.7820436083796494
0.7819767441860465
0.7819953834316491
0.7820140194905112
0.7820326523634499
0.782051282051282
0.7819844457738655
0.782003076397197
0.7820217038366231
0.7820403280929596
0.7820589491670227
0.7820775670596275
0.7820961817715897
0.7821147933037239
0.7821334016568452
0.7821520068317677
0.7821706088293058
0.7821892076502732
0.7822078032954837
0.7822263957657504
0.7821596244131456
0.7821782178217822
0.7821968080566698
0.7822153951186209
0.7822339790084478
0.7822525597269625
0.7822711372749765
0.7822897116533015
0.7823082828627484
0.7823268509041282
0.7823454157782516
0.7823639774859287
0.7823825360279696
0.7823158253751705
0.7822491260977066
0.7821824381926684
0.7822010058818515
0.7821343334469826
0.7821529020710816
0.7820862450997103
0.782019599488709
0.7820381731424676
0.7820567436312515
0.7820753109558698
0.7820938751171309
0.7821124361158432
0.7821309939528149
0.7821495486288537
0.7821681001447671
0.7821866485013624
0.7821200510855684
0.7821386003745

0.7829591249280369
0.7829769736842105
0.782994819504975
0.783012662391054
0.783030502343172
0.7829661295626439
0.7829017673653925
0.7829196120335361
0.7829374537683899
0.7829552925706772
0.7829731284411209
0.7829909613804437
0.7830087913893682
0.7830266184686165
0.7830444426189107
0.7830622638409725
0.7830800821355236
0.7830978975032852
0.7831157099449783
0.7831335194613237
0.7831513260530422
0.7830870279146142
0.7831048353993925
0.7831226399605976
0.7831404415989494
0.7831582403151675
0.7831760361099713
0.7831938289840801
0.7832116189382129
0.7831473580571053
0.7831651489047502
0.7831829368334701
0.7831186941186121
0.7830544619422573
0.7829902403018125
0.78300803673938
0.7829438294382944
0.7829616267628731
0.78297942116914
0.7829972126578127
0.783015001229609
0.7830327868852459
0.7830505696254405
0.7829863956728406
0.7830041793001721
0.7830219600131104
0.783039737812372
0.7830575126986727
0.7830752846727288
0.7830930537352556
0.7831108198869686
0.7831285831285831
0.783064450085988
0.7

0.7838717358520517
0.7838888888888889
0.7839060392032379
0.7839231867957467
0.7839403316670635
0.7839574738178356
0.7838952796509322
0.7839124226558781
0.7839295629412232
0.7839467005076142
0.7839638353556984
0.7839016653449643
0.7839188010467053
0.7839359340310815
0.7839530642987393
0.783970191850325
0.7839873166864844
0.7840044388078631
0.7839423000713324
0.7839594230464416
0.7838972977256518
0.7839144215530903
0.7839315426669836
0.7839486610679766
0.783965776756714
0.7839828897338403
0.783920792079208
12625
tensor(9897)


(12625, tensor(9897))